In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np

In [3]:
mtcnn = MTCNN(image_size=160, margin=16)
resnet = InceptionResnetV1(pretrained='vggface2').eval()
import cv2
face_cascade = cv2.CascadeClassifier('./cascade.xml')

In [4]:
from torch.utils.data import Dataset, SubsetRandomSampler
import os
from PIL import Image, ImageOps, ImageStat, ImageEnhance
from torchvision import transforms
import torch
import numpy as np
import cv2
import torch.nn as nn
import torch.optim as optim

def brightness( im_file ):
   im = Image.fromarray(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]


class LFW(Dataset):
    def __init__(self, path_to_data, transform=None):
        self.transform = transform
        data_same_person = []
        data_dif_person = []
        with open(path_to_data, 'r') as f:
            line = f.read()
            line = line.split('\n')
            line = [i.split('\t') for i in line]
            [data_same_person.append(i) if len(i) == 3 else data_dif_person.append(i) for i in line]
        data_same_person_res = []
        for i in data_same_person:
            if len(i[1]) == 1:
                inp = f'./lfw/{i[0]}/{i[0]}_000{i[1]}.jpg'
            elif len(i[1]) ==2:
                inp = f'./lfw/{i[0]}/{i[0]}_00{i[1]}.jpg'
            else:
                inp = f'./lfw/{i[0]}/{i[0]}_0{i[1]}.jpg'
            if len(i[2]) == 1:
                tar = f'./lfw/{i[0]}/{i[0]}_000{i[2]}.jpg'
            elif len(i[2]) ==2:
                tar = f'./lfw/{i[0]}/{i[0]}_00{i[2]}.jpg'
            else:
                tar = f'./lfw/{i[0]}/{i[0]}_0{i[2]}.jpg'
            data_same_person_res.append([inp, tar, 1.0])

        #data_same_person_res = [[f'./lfw/{i[0]}/{i[0]}_000{i[1]}.jpg' if len(i[1]) == 1 else f'./lfw/{i[0]}/{i[0]}_00{i[1]}.jpg',
                                 #f'./lfw/{i[0]}/{i[0]}_000{i[2]}.jpg' if len(i[2]) == 1 else f'./lfw/{i[0]}/{i[0]}_00{i[2]}.jpg',
                                 #1.0] for i in data_same_person]
        del (data_dif_person[-1])
        data_dif_person_res = []
        for i in data_dif_person:
            if len(i[1]) == 1:
                inp = f'./lfw/{i[0]}/{i[0]}_000{i[1]}.jpg'
            elif len(i[1]) ==2:
                inp = f'./lfw/{i[0]}/{i[0]}_00{i[1]}.jpg'
            else:
                inp = f'./lfw/{i[0]}/{i[0]}_0{i[1]}.jpg'
            if len(i[3]) == 1:
                tar = f'./lfw/{i[2]}/{i[2]}_000{i[3]}.jpg'
            elif len(i[3]) ==2:
                tar = f'./lfw/{i[2]}/{i[2]}_00{i[3]}.jpg'
            else:
                tar = f'./lfw/{i[2]}/{i[2]}_0{i[3]}.jpg'
            data_dif_person_res.append([inp, tar, 0.0])
        #data_dif_person_res = [[f'./lfw/{i[0]}/{i[0]}_000{i[1]}.jpg' if len(i[1]) == 1 else f'./lfw/{i[0]}/{i[0]}_00{i[1]}.jpg',
        #                        f'./lfw/{i[2]}/{i[2]}_000{i[3]}.jpg' if len(i[3]) == 1 else f'./lfw/{i[2]}/{i[2]}_00{i[3]}.jpg',
        #                        0.0] for i in data_dif_person]
        self.data = data_same_person_res + data_dif_person_res

    def __len__(self):
        return len(self.data)

    '''def __getitem__(self, index):
        img_input = mtcnn(Image.open(self.data[index][0])).detach().numpy()
        img_target = mtcnn(Image.open(self.data[index][1])).detach().numpy()
        label = float(self.data[index][2])
        return img_input, img_target, label'''

    def __getitem__(self, index):
        img_inp = cv2.imread(self.data[index][0])
        img = img_inp.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            faces = img_inp[y:y + h, x:x + w]
        img_input = faces
        img_input = cv2.cvtColor(img_input, cv2.COLOR_BGR2GRAY)
        img_input = controller(img_input, 60)



        test = Image.fromarray(img_input)
        test = ImageOps.grayscale(test)

        stat = ImageStat.Stat(test)
        stat = stat.mean[0]
        filter = ImageEnhance.Brightness(test)
        image = filter.enhance(100/brightness(img_input))
        image = np.asarray(image)
        image = cv2.merge([image, image, image])
        image = np.reshape(image, (3, image.shape[0], image.shape[1]))
        image = np.resize(image, (3, 160, 160))


        tar = cv2.imread(self.data[index][1])
        img = tar.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            faces = tar[y:y + h, x:x + w]
        img_target = faces
        img_target = cv2.cvtColor(img_target, cv2.COLOR_BGR2GRAY)
        img_target = cv2.merge([img_target, img_target, img_target])
        img_target = np.reshape(img_target, (3, img_target.shape[0], img_target.shape[0]))
        img_target = np.resize(img_target, (3, 160, 160))
        label = float(self.data[index][2])
        return image, img_target, label


class LFW_Train(Dataset):
    def __init__(self, path_to_data, transform=None):
        self.transform = transform
        data_same_person = []
        data_dif_person = []
        with open(path_to_data, 'r') as f:
            line = f.read()
            line = line.split('\n')
            line = [i.split('\t') for i in line]
            [data_same_person.append(i) if len(i) == 3 else data_dif_person.append(i) for i in line]
        res = []
        for i in data_same_person:
            for j in data_dif_person:
                if i[0] in j:
                    index = j.index(i[0])
                    if index == 0:
                        index = 2
                    else:
                        index = 0
                    tmp = i.copy()
                    tmp.append(j[index])
                    tmp.append(j[index+1])
                    res.append(tmp)
        data_res = []
        for i in res:
            if len(i[1]) == 1:
                anchor = f'./lfw/{i[0]}/{i[0]}_000{i[1]}.jpg'
            elif len(i[1]) == 2:
                anchor = f'./lfw/{i[0]}/{i[0]}_00{i[1]}.jpg'
            else:
                anchor = f'./lfw/{i[0]}/{i[0]}_0{i[1]}.jpg'
            if len(i[2]) == 1:
                pos = f'./lfw/{i[0]}/{i[0]}_000{i[2]}.jpg'
            elif len(i[2]) == 2:
                pos = f'./lfw/{i[0]}/{i[0]}_00{i[2]}.jpg'
            else:
                pos = f'./lfw/{i[0]}/{i[0]}_0{i[2]}.jpg'
            if len(i[4]) == 1:
                neg = f'./lfw/{i[3]}/{i[3]}_000{i[4]}.jpg'
            elif len(i[4]) == 2:
                neg = f'./lfw/{i[3]}/{i[3]}_00{i[4]}.jpg'
            else:
                neg = f'./lfw/{i[3]}/{i[3]}_0{i[4]}.jpg'
            data_res.append([anchor, pos, neg])
        self.data = data_res

    def __len__(self):
        return len(self.data)

    '''def __getitem__(self, index):
        img_input = mtcnn(Image.open(self.data[index][0])).detach().numpy()
        img_target = mtcnn(Image.open(self.data[index][1])).detach().numpy()
        label = float(self.data[index][2])
        return img_input, img_target, label'''

    ''' def __getitem__(self, index):
        img_inp = Image.open(self.data[index][0])
        img_inp = np.asarray(ImageOps.grayscale(img_inp))
        img_inp = cv2.merge([img_inp, img_inp, img_inp])
        anchor = mtcnn(img_inp).detach().numpy()

        pos = Image.open(self.data[index][1])
        pos = np.asarray(ImageOps.grayscale(pos))
        pos = cv2.merge([pos, pos, pos])
        pos = mtcnn(pos).detach().numpy()

        tar = Image.open(self.data[index][2])
        tar = np.asarray(ImageOps.grayscale(tar))
        tar = cv2.merge([tar, tar, tar])
        neg = mtcnn(tar).detach().numpy()

        return anchor, pos, neg'''

    def __getitem__(self, index):


        img_inp = cv2.imread(self.data[index][0])
        img = img_inp.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            faces = img_inp[y:y + h, x:x + w]
        anchor = faces
        #print(self.data[index][0])
        anchor = cv2.cvtColor(anchor, cv2.COLOR_BGR2GRAY)


        img_inp = cv2.imread(self.data[index][1])
        img = img_inp.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            faces = img_inp[y:y + h, x:x + w]
        pos = faces
        #print(self.data[index][1])
        pos = cv2.cvtColor(pos, cv2.COLOR_BGR2GRAY)
        pos = cv2.merge([pos, pos, pos])
        pos = np.reshape(pos, (3, pos.shape[0], pos.shape[0]))

        img_inp = cv2.imread(self.data[index][2])
        img = img_inp.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            faces = img_inp[y:y + h, x:x + w]
        neg = faces
        #print(self.data[index][2])
        neg = cv2.cvtColor(neg, cv2.COLOR_BGR2GRAY)
        neg = cv2.merge([neg, neg, neg])
        neg = np.reshape(neg, (3, neg.shape[0], neg.shape[0]))


        if self.transform:
            anchor = self.transform(image=anchor)['image']
            anchor = controller(anchor)
            anchor = cv2.merge([anchor, anchor, anchor])
            anchor = np.reshape(anchor, (3, anchor.shape[0], anchor.shape[0]))
        anchor = np.resize(anchor, (3, 160, 160))
        pos = np.resize(pos, (3, 160, 160))
        neg = np.resize(neg, (3, 160, 160))
        return np.array(anchor, dtype=np.float), np.array(pos, dtype=np.float), np.array(neg, dtype=np.float)



In [5]:
import random
def controller(img, brightness=40, contrast=127):
    p = random.randint(1, 3)
    if p != 2:
        return img
    p = random.randint(1, 5)
    if p == 1:
        brightness = 10
    elif p == 2:
        brightness = 20
    elif p == 3:
        brightness = 40
    elif p == 4:
        brightness == 60
    else:
        brightness = 80
    brightness = int((brightness - 0) * (255 - (-255)) / (510 - 0) + (-255))
    contrast = int((contrast - 0) * (127 - (-127)) / (254 - 0) + (-127))
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            max = 255
        else:
            shadow = 0
            max = 255 + brightness
        al_pha = (max - shadow) / 255
        ga_mma = shadow
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(img, al_pha,
                              img, 0, ga_mma)
    else:
        cal = img
    if contrast != 0:
        Alpha = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        Gamma = 127 * (1 - Alpha)
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(cal, Alpha,
                              cal, 0, Gamma)
    return cal

In [6]:
import albumentations as A

train_dataset = LFW_Train("./pairsDevTrain.txt",
                        transform=A.Compose([
                                A.HorizontalFlip(p=0.5),
                                A.Rotate(limit=(0, 10), p=0.5)
                            ])
                      )
test_dataset = LFW("./pairsDevTest.txt"
                      )

In [7]:
device = torch.device("cuda:0")
resnet = resnet.to(device)
batch_size = 32

data_size = len(train_dataset)
validation_fraction = .2


val_split = int(np.floor((validation_fraction) * data_size))
indices = list(range(data_size))
np.random.seed(42)
np.random.shuffle(indices)

val_indices, train_indices = indices[:val_split], indices[val_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [8]:
from sklearn.metrics import f1_score
def compute_f1(model, grt):
    model.eval()
    for x, y, z in grt:
        x = x.to(device=device)
        y = y.to(device=device)
        z = z.to(device=device)
        predictx = model(x.float())
        predicty = model(y.float())
        predictz = model(z.float())
        pred = []
        label = []
        for i in range(len(predictx)):
            pred.append((1+sum(predictx[i]*predicty[i]))/2)
            pred.append((1+sum(predictx[i]*predictz[i]))/2)
            label.append(1.0)
            label.append(0.0)
        pred = np.array(list(map(lambda x: int(x > 0.8), pred)))
        label = np.array(label)
        return f1_score(label, pred)

def compute_f1_test(model, grt):
    model.eval()
    for x, y, label in grt:
        x = x.to(device=device)
        y = y.to(device=device)
        predictx = model(x.float())
        predicty = model(y.float())
        pred = []
        for i in range(len(predictx)):
            pred.append((1+sum(predictx[i]*predicty[i]))/2)
        pred = np.array(list(map(lambda x: int(x > 0.8), pred)))
        return f1_score(label, pred)

In [9]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):
    loss_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        loss_accum = 0
        for i_step, (a, p, n) in enumerate(train_loader):
            a_gpu = torch.tensor(a, dtype=torch.float).to(device)
            p_gpu = torch.tensor(p, dtype=torch.float).to(device)
            n_gpu = torch.tensor(n, dtype=torch.float).to(device)
            a_gpu = a_gpu.requires_grad_(True)
            p_gpu = p_gpu.requires_grad_(True)
            n_gpu = n_gpu.requires_grad_(True)
            a_gpu = model(a_gpu)
            p_gpu = model(p_gpu)
            n_gpu = model(n_gpu)
            #a_gpu.requires_grad = True
            #p_gpu.requires_grad = True
            #n_gpu.requires_grad = True
            loss_value = loss(a_gpu, p_gpu, n_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        val_f1 = compute_f1(model, val_loader)

        loss_history.append(float(ave_loss))
        val_history.append(val_f1)

        print("%f Average loss: %f,, Val f1: %f" % (epoch, ave_loss, val_f1))

    return loss_history, val_history

In [10]:
import warnings
params_old = list([param for name, param in resnet.named_parameters() if not name.startswith('logits.')])
params_new = list([param for name, param in resnet.named_parameters() if name.startswith('logits.')])

parameters = [
    {'params': params_old, 'lr': 0.0001, 'momentum': 0.9},
    {'params': params_new, 'lr': 0.001, 'momentum': 0.9}
]
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
loss = nn.TripletMarginLoss(margin=1.0, p=2)
warnings.filterwarnings("ignore")
train_model(resnet, train_loader, val_loader, loss, optimizer, 100)
compute_f1_test(resnet, test_loader)

0.000000 Average loss: 1.019784,, Val f1: 0.355556
1.000000 Average loss: 0.962874,, Val f1: 0.468085
2.000000 Average loss: 0.868221,, Val f1: 0.444444
3.000000 Average loss: 0.931090,, Val f1: 0.408163
4.000000 Average loss: 0.965285,, Val f1: 0.375000
5.000000 Average loss: 0.871385,, Val f1: 0.531250
6.000000 Average loss: 0.799550,, Val f1: 0.695652
7.000000 Average loss: 0.901166,, Val f1: 0.557377
8.000000 Average loss: 0.800386,, Val f1: 0.600000
9.000000 Average loss: 0.800010,, Val f1: 0.566667
10.000000 Average loss: 0.820342,, Val f1: 0.600000
11.000000 Average loss: 0.756162,, Val f1: 0.571429
12.000000 Average loss: 0.767751,, Val f1: 0.636364
13.000000 Average loss: 0.720604,, Val f1: 0.633333
14.000000 Average loss: 0.651553,, Val f1: 0.617647
15.000000 Average loss: 0.725965,, Val f1: 0.634921
16.000000 Average loss: 0.756293,, Val f1: 0.526316
17.000000 Average loss: 0.836907,, Val f1: 0.533333
18.000000 Average loss: 0.722676,, Val f1: 0.615385
19.000000 Average loss

0.6938775510204082

In [ ]:
torch.save(resnet.state_dict(), 'm2.w')

In [10]:
import pickle
with open('pairs_labled.pkl', 'rb') as f:
    data = pickle.load(f)

In [11]:
import cv2
import torch
resnet.load_state_dict(torch.load('./m1.w'))
resnet.eval()
resnet.to('cpu')
x = []
y = []
label = []
predictx = []
predicty = []
for i in data:
    image = cv2.merge([i[0], i[0], i[0]])
    image = np.reshape(image, (3, image.shape[0], image.shape[1]))
    image = np.resize(image, (3, 160, 160))
    x = torch.Tensor(np.array([image])).to('cpu')
    predictx.append(resnet(x.float()).detach().numpy()[0])
    target = cv2.merge([i[1], i[1], i[1]])
    target = np.reshape(target, (3, target.shape[0], target.shape[1]))
    target = np.resize(target, (3, 160, 160))
    y = torch.Tensor(np.array([target])).to('cpu')
    predicty.append(resnet(y.float()).detach().numpy()[0])

    label.append(i[2])
pred = []
for i in range(len(predictx)):
    pred.append((1+sum(predictx[i]*predicty[i]))/2)

pred = np.array(list(map(lambda x: int(x > 0.8), pred)))
print(f1_score(label, pred))

0.5254237288135594
